In [18]:
from Bio import AlignIO
from Bio.Alphabet import generic_dna, generic_protein, single_letter_alphabet
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
import numpy as np

In [8]:
aln_file = '/home/sur/micropopgen/exp/2018/today9/aln/dnaG.aln'
aln = AlignIO.read(aln_file, 'fasta')
print(aln)

SingleLetterAlphabet() alignment with 3 rows and 730 columns
----MYKLSEDEEKIINNIDIVDLIGQYVDLNKAGVSYKGYSPF...KGD Streptobacillus_moniliformis_61686.CDS_0
-----VIDQQTRQRILDSAQILDVVSDFISLRRQGVSYVGLCPF...IG- Porphyromonas_sp_57899.CDS_0
MTTGHFLPPEFLDELRQRITLSSLIGRTVKLTRAGREFRACCPF...EG- Zymomonas_mobilis_57566.CDS_0


In [ ]:
def filter_alignment(aln, gap_prop=0.99, remove_singletons=True):

In [9]:
def align2array(aln):
    """Convert multiple sequence alignment object to numpy array.
    Taken from tutorial."""
    
    a_array = np.array([list(rec) for rec in aln], np.character)
    
    return(a_array)

In [10]:
def array2align(arr, names, alphabet):
    """Convert numpy array to multiple sequence alignment.
    Adapted from documentation"""
    
    records = []
    
    # Iterate over array rows (i.e. records)
    for i in range(arr.shape[0]):
        seq = ''.join(np.array(arr[i], dtype=str))
        name = names[i]
        
        # Concatenate sequence records
        records.append(SeqRecord(Seq(seq, alphabet), id=name))
    
    # Convert to MSA
    new_aln = MultipleSeqAlignment(records)
    
    return(new_aln)

In [13]:
arr = align2array(aln)
arr.shape

(3, 730)

In [21]:
aln2 = array2align(arr, [r.id for r in aln], generic_protein)
print(aln2)

ProteinAlphabet() alignment with 3 rows and 730 columns
----MYKLSEDEEKIINNIDIVDLIGQYVDLNKAGVSYKGYSPF...KGD Streptobacillus_moniliformis_61686.CDS_0
-----VIDQQTRQRILDSAQILDVVSDFISLRRQGVSYVGLCPF...IG- Porphyromonas_sp_57899.CDS_0
MTTGHFLPPEFLDELRQRITLSSLIGRTVKLTRAGREFRACCPF...EG- Zymomonas_mobilis_57566.CDS_0


In [ ]:
print(a_array)

In [ ]:
nseqs = len(a)
index = np.ones(a_array.shape[1], dtype=int)
for i in range(a_array.shape[1]):
    c = a_array[:,i]
    counts = np.unique(c, return_counts=True)
    
    # Remove unique columns
    if counts[0].shape == (1,):
        index[i] = 0
        continue
    
    # Count gaps
    ngaps = counts[1][b'-' == counts[0]]
    if ngaps / nseqs > 0.2:
        index[i] = 0
        continue
    
#     print(c)
index = np.array(index, dtype=bool)
filtered = a_array[:,index]

In [ ]:
print(a_array.shape)
print(filtered.shape)

In [ ]:
cols_to_keep = np.argwhere(index)
new_aln = a[:,int(cols_to_keep[0])]
itercols = iter(cols_to_keep)
next(itercols)
for i in itercols:
    new_aln = new_aln + a[:,i]
    #print(a[:,int(i)])
    
    
    

In [ ]:
cols_to_keep = np.argwhere(index)
# print(cols_to_keep)
# print(a[:,0:5])
# print(a[:,0])
a[:,int(cols_to_keep[0])]

In [ ]:
AlignIO.MultipleSeqAlignment?

In [ ]:
a = a_array[:,0:5]
i = np.array([1,1,1,1,0], dtype=bool)
print(i)
print(a)
print(a[:,i])

In [ ]:
np.array(i, dtype=int)

In [ ]:
a[0] == b'-'

In [ ]:
type(a[0] == b'-')

In [ ]:
counts[0][0]

In [ ]:
counts


In [ ]:
counts[1][b'S' == counts[0]]